## Barrel Shifter

A Barrel shifter performs the operation `v << s` (or `v >> s` with flipped bits) where `s` is not constant.
An elegant implementation is the approach via cascaded multiplexer elements.

This `cshift` primitive is later instanced procedurally, iterating through the variable `i` 

### Shifter element in MyHDL

In [1]:
from myirl.emulation.myhdl import *

@block
def cshift(q : Signal.Output, a : Signal, b : Signal, sbit : Signal, 
           msb : Signal,
           # These can take a signal or a fixed bool:
           asr, rotate,
           WRAP : bool):
    carry = Signal(bool()) # Carry bit
    u, v = [ Signal(bool()) for i in range(2) ]
    
    @always_comb
    def assign_carry():
        if asr: # arithmetic shift right
            carry.next = sbit & msb
        else:
            carry.next = 0

    @always_comb
    def assign():
        u.next = a & ~sbit
        
        if rotate == False:         
            if WRAP:
                v.next = carry
            else:
                v.next = b & sbit
        else:
            v.next = b & sbit
            
    @always_comb
    def assign_q():    
        q.next = u | v
        
    return instances()

### Shifter stage

In [2]:
@block
def shifter_stage(
    shifter,
    w_in : Signal,
    w_out : Signal.Output,
    msb : Signal,
    nmux : int, sbit : Signal, DATA_WIDTH : int, W_POWER : int, asr : bool, rotate : bool
):
    inst = []

    # Create signal array
    w = [ Signal(bool()) for i in range(DATA_WIDTH) ]
    wo = concat(*reversed(w))
    wi = [ w_in[i] for i in range(DATA_WIDTH) ]

    MUX_W = DATA_WIDTH // nmux

    for imux in range(nmux):
        tmp = imux * MUX_W
        # print(imux)
        for i in range(tmp, tmp + MUX_W):
            j = i + MUX_W//2
            m = j % DATA_WIDTH
            inst.append(shifter(w[m], wi[m], wi[i], sbit, msb, asr, rotate,
                         j >= DATA_WIDTH ))

    @always_comb
    def assign():
        w_out.next = wo

    return instances()

### The barrel shifter implementation

In [3]:
@block
def barrel_shifter(shifter, clk : ClkSignal, ce : Signal, val : Signal, s : Signal, result : Signal.Output, \
                   rotate = False, W_POWER = 5 ):
    
    DATA_WIDTH = 2 ** W_POWER
    print("DATA WIDTH", DATA_WIDTH, "ROTATE", rotate)
        
    worker = [ val ]
    worker = worker + [ Signal(intbv()[DATA_WIDTH:]) for i in range(W_POWER) ]
    msb = val[DATA_WIDTH-1]

    sbit = [ s[i] for i in range(len(s))]
    
    shifter_stages = []
    for stage in range(W_POWER):
        K = W_POWER - stage - 1
        print("Stage %d" % stage)
        shifter_stages.append( \
                shifter_stage(shifter, worker[stage], worker[stage + 1], msb, 2 ** stage, sbit[K], \
                            DATA_WIDTH, W_POWER, False, rotate) \
                             )
        
    @always(clk.posedge)
    def assign():
        if ce == True:
            result.next = worker[W_POWER]
        
    return instances()

### Translate and test

In [4]:
W_POWER = 4

from myirl.test.common_test import gen_osc
import myirl

@block
def top_bs(shifter):
    clk = ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        barrel_shifter(shifter, clk, ce, val, s, result, False, W_POWER),
        gen_osc(clk, 2)
    ]
    
    
    TEST_VALUES = [
        (0xdead, 8, 0xad00),
        (0x8f01, 15, 0x8000),
    ]

    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == item[2]

    
    inst += [ stim ]
    return inst
 
@utils.timer
def test(shifter_element):    
    return top_bs(shifter_element)

design = test(cshift)

DATA WIDTH 16 ROTATE False
Stage 0
 Module top_top_bs: Existing instance cshift, rename to cshift_1 
Stage 1
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_1 
Stage 2
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_2 
Stage 3
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_3 
Creating process 'barrel_shifter/assign' with sensitivity (clk'rising,)
Creating sequential 'top_bs/stim' 
Finished test in 1.4497 secs


In [5]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
# print(f)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 Elaborating component shifter_stage__wrapped_wrapper_s16_s16_s1_8_s1_16_4_0_0 
 Writing 'shifter_stage_3' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_3.vhdl 
 Elaborating component shifter_stage__wrapped_wrapper_s16_s16_s1_4_s1_16_4_0_0 
 Writing 'shifter_stage_2' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_2.vhdl 
 Elaborating component shifter_stage__wrapped_wrapper_s16_s16_s1_2_s1_16_4_0_0 
 Writing 'shifter_stage_1' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_1.vhdl 
 Elaborating component cshift_s1_s1_s1_s1_s1_0_0_1 
 Writing 'cshift_1' to file /tmp/myirl_top_top_bs_oe45hckw/cshift_1.vhdl 
 Elaborating component cshift_s1_s1_s1_s1_s1_0_0_0 
 Writing 'cshift' to file /tmp/myirl_top_top_bs_oe45hckw/cshift.vhdl 
 Elaborating component shifter_stage__wrapped_wrapper_s16_s16_s1_1_s1_16_4_0_0 
 Writing 'shifter_stage' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage.vhdl 
 Elaborating component barrel_shifter__wrapped_wrapper_s1_s1_s16_s4_s16_0_4 
 Writin

0

In [6]:
# !cat {f[-3]}

## Optimization

By implementing the often called `cshift` unit in IRL, we save a few cycles.

In [7]:
c = myirl.LibraryModule("bs", targets.VHDL, debug = False)

@myirl.component(c)
def cshift_irl(q : Signal.Output, a : Signal, b : Signal, sbit : Signal, 
           msb : Signal,
           # These can take a signal or a fixed bool:
           asr, rotate,
           WRAP : bool):
    carry = Signal(bool(), name = "carry") # Carry bit
    u, v = [ Signal(bool(), name = n) for n in "uv" ]
    
    
    @myirl.genprocess()
    def selector():
        If = selector.If
        yield [
            If(asr == True).Then(
                carry.set(sbit & msb)
            ).Else(
                carry.set(False)
            ),

            If(rotate == False).Then(
                If(WRAP == True).Then(
                    v.set(carry)
                ).Else(
                    v.set(b & sbit)
                )
            ).Else(
                v.set(b & sbit)
            ),    
        ]
    
    wires = [
        u.set(a & ~sbit),
        q.set(u | v)
    ]
        
    return locals()

 Declare obj 'cshift_irl' in context '(LIB: LibraryModule 'bs')' 


In [8]:
design = test(cshift_irl)

 Module top_top_bs: Existing instance top_bs, rename to top_bs_1 
 Module top_top_bs: Existing instance barrel_shifter, rename to barrel_shifter_1 
DATA WIDTH 16 ROTATE False
Stage 0
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_4 
 Module bs: Existing instance cshift_irl, rename to cshift_irl_1 
Stage 1
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_5 
Stage 2
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_6 
Stage 3
 Module top_top_bs: Existing instance shifter_stage, rename to shifter_stage_7 
Creating process 'barrel_shifter/assign' with sensitivity (clk'rising,)
Creating sequential 'top_bs/stim' 
Finished test in 0.3276 secs


In [9]:
f = design.elab(targets.VHDL, elab_all = True)
f += c.elab(targets.VHDL)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = False)

 Elaborating component shifter_stage__ComponentObj_s16_s16_s1_8_s1_16_4_0_0 
 Writing 'shifter_stage_7' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_7.vhdl 
 Elaborating component shifter_stage__ComponentObj_s16_s16_s1_4_s1_16_4_0_0 
 Writing 'shifter_stage_6' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_6.vhdl 
 Elaborating component shifter_stage__ComponentObj_s16_s16_s1_2_s1_16_4_0_0 
 Writing 'shifter_stage_5' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_5.vhdl 
 Elaborating component shifter_stage__ComponentObj_s16_s16_s1_1_s1_16_4_0_0 
 Writing 'shifter_stage_4' to file /tmp/myirl_top_top_bs_oe45hckw/shifter_stage_4.vhdl 
 Elaborating component barrel_shifter__ComponentObj_s1_s1_s16_s4_s16_0_4 
 Writing 'barrel_shifter_1' to file /tmp/myirl_top_top_bs_oe45hckw/barrel_shifter_1.vhdl 
 Elaborating component top_bs__ComponentObj 
 Writing 'top_bs_1' to file /tmp/myirl_top_top_bs_oe45hckw/top_bs_1.vhdl 
 Elaborating component shifter_stage__wrapped_wrapper_s16

0

## Optimization

A bit more optimization can be done by moving the code into a cythonizeable library. See
[Cythonized barrel shifter](example_cython_barrelshifter.ipynb)